In [ ]:
# Import Libraries 
import cv2
import numpy as np
import torch
import torch.nn as nn
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image
import torchvision.transforms as transforms
from ultralytics import YOLO
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [ ]:
# Load Models 
# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"🎯 Using device: {device}")

# 1. Load License Plate Detector (YOLO)
plate_detector_path = Path('runs/detect/license_plate_v1/weights/best.pt')
if not plate_detector_path.exists():
    print(f"❌ Plate detector not found at {plate_detector_path}")
    raise FileNotFoundError("Please train the plate detector first (Notebook 2)")

plate_detector = YOLO(str(plate_detector_path))
print(f"✅ License plate detector loaded")

# 2. Load Character Recognition Model
char_model_path = Path('character_recognition_best.pth')
if not char_model_path.exists():
    print(f"❌ Character model not found at {char_model_path}")
    raise FileNotFoundError("Please train the character recognition model first (Notebook 3)")

# Load checkpoint
checkpoint = torch.load(char_model_path, map_location=device)
CHAR_TO_IDX = checkpoint['char_to_idx']
IDX_TO_CHAR = checkpoint['idx_to_char']
NUM_CLASSES = len(CHAR_TO_IDX)

# Define model architecture (same as training)
class CharacterCNN(nn.Module):
    def __init__(self, num_classes=31):
        super(CharacterCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dropout1 = nn.Dropout2d(0.25)
        
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.bn4 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dropout2 = nn.Dropout2d(0.25)
        
        self.conv5 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn5 = nn.BatchNorm2d(128)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, padding=1)
        self.bn6 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout3 = nn.Dropout2d(0.25)
        
        self.fc1 = nn.Linear(128 * 3 * 3, 512)
        self.bn7 = nn.BatchNorm1d(512)
        self.dropout4 = nn.Dropout(0.5)
        
        self.fc2 = nn.Linear(512, 256)
        self.bn8 = nn.BatchNorm1d(256)
        self.dropout5 = nn.Dropout(0.5)
        
        self.fc3 = nn.Linear(256, num_classes)
    
    def forward(self, x):
        import torch.nn.functional as F
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool1(x)
        x = self.dropout1(x)
        
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool2(x)
        x = self.dropout2(x)
        
        x = F.relu(self.bn5(self.conv5(x)))
        x = F.relu(self.bn6(self.conv6(x)))
        x = self.pool3(x)
        x = self.dropout3(x)
        
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.bn7(self.fc1(x)))
        x = self.dropout4(x)
        
        x = F.relu(self.bn8(self.fc2(x)))
        x = self.dropout5(x)
        
        x = self.fc3(x)
        return x

# Load model
char_model = CharacterCNN(num_classes=NUM_CLASSES).to(device)
char_model.load_state_dict(checkpoint['model_state_dict'])
char_model.eval()
print(f"✅ Character recognition model loaded")

# Transform for character images
char_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

print(f"\n📊 Models Summary:")
print(f"   Plate Detector: YOLOv8")
print(f"   Character Model: CNN with {NUM_CLASSES} classes")
print(f"   Characters: {list(CHAR_TO_IDX.keys())}")

🎯 Using device: cuda
❌ Plate detector not found at runs\detect\license_plate_v1\weights\best.pt


FileNotFoundError: Please train the plate detector first (Notebook 2)

In [3]:
# Character Segmentation Functions 
def preprocess_plate_for_ocr(plate_img):
    """
    Tiền xử lý ảnh biển số để tách ký tự
    """
    if len(plate_img.shape) == 3:
        gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
    else:
        gray = plate_img.copy()
    
    # Resize
    h, w = gray.shape
    new_h = 64
    new_w = int(w * new_h / h)
    gray = cv2.resize(gray, (new_w, new_h))
    
    # Convert to HSV
    plate_hsv = cv2.cvtColor(cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR), cv2.COLOR_BGR2HSV)
    v_channel = plate_hsv[:, :, 2]
    
    # Adaptive threshold
    binary = cv2.adaptiveThreshold(
        v_channel, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        blockSize=21,
        C=10
    )
    
    # Morphological operations
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
    
    return binary, gray

def segment_characters(binary_img, original_img):
    """
    Tách các ký tự từ ảnh nhị phân
    """
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    char_contours = []
    h, w = binary_img.shape
    
    for contour in contours:
        x, y, cw, ch = cv2.boundingRect(contour)
        
        aspect_ratio = cw / ch if ch > 0 else 0
        area = cv2.contourArea(contour)
        bbox_area = cw * ch
        solidity = area / bbox_area if bbox_area > 0 else 0
        
        if (0.15 < aspect_ratio < 1.0 and
            ch > h * 0.3 and
            ch < h * 0.9 and
            cw > 5 and
            solidity > 0.3):
            char_contours.append((x, y, cw, ch, contour))
    
    # Sort by x coordinate
    char_contours = sorted(char_contours, key=lambda c: c[0])
    
    # Extract character images
    char_images = []
    for x, y, cw, ch, contour in char_contours:
        padding = 2
        x1 = max(0, x - padding)
        y1 = max(0, y - padding)
        x2 = min(w, x + cw + padding)
        y2 = min(h, y + ch + padding)
        
        char_img = original_img[y1:y2, x1:x2]
        char_img = cv2.resize(char_img, (28, 28))
        
        char_images.append({
            'image': char_img,
            'bbox': (x, y, cw, ch),
            'position': x
        })
    
    return char_images

print("✅ Character segmentation functions defined")

✅ Character segmentation functions defined


In [ ]:
# End-to-End Recognition Pipeline 
class LicensePlateRecognizer:
    """
    Complete License Plate Recognition Pipeline
    """
    def __init__(self, plate_detector, char_model, char_transform, device):
        self.plate_detector = plate_detector
        self.char_model = char_model
        self.char_transform = char_transform
        self.device = device
        
    def detect_plate(self, image_path, conf_threshold=0.25):
        """
        Detect license plate in image
        """
        results = self.plate_detector.predict(
            source=str(image_path),
            conf=conf_threshold,
            verbose=False
        )
        
        detections = []
        if len(results[0].boxes) > 0:
            for box in results[0].boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                detections.append({
                    'bbox': (x1, y1, x2, y2),
                    'confidence': conf
                })
        
        return detections
    
    def recognize_characters(self, char_images):
        """
        Recognize characters using CNN model
        """
        if len(char_images) == 0:
            return []
        
        # Prepare batch
        batch = []
        for char_data in char_images:
            char_img = char_data['image']
            # Convert to tensor
            char_tensor = self.char_transform(char_img)
            batch.append(char_tensor)
        
        batch = torch.stack(batch).to(self.device)
        
        # Predict
        with torch.no_grad():
            outputs = self.char_model(batch)
            probs = torch.softmax(outputs, dim=1)
            confidences, predictions = torch.max(probs, dim=1)
        
        # Get characters
        recognized_chars = []
        for i, (pred, conf) in enumerate(zip(predictions, confidences)):
            char = IDX_TO_CHAR[pred.item()]
            recognized_chars.append({
                'char': char,
                'confidence': conf.item(),
                'position': char_images[i]['position']
            })
        
        return recognized_chars
    
    def process_image(self, image_path, visualize=False):
        """
        Complete pipeline: detect plate -> segment chars -> recognize
        """
        # Read image
        img = cv2.imread(str(image_path))
        if img is None:
            return None
        
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Detect plate
        plate_detections = self.detect_plate(image_path)
        
        if len(plate_detections) == 0:
            return {
                'success': False,
                'message': 'No license plate detected',
                'image': img_rgb
            }
        
        # Get best detection
        best_detection = max(plate_detections, key=lambda x: x['confidence'])
        x1, y1, x2, y2 = best_detection['bbox']
        
        # Crop plate
        plate = img[y1:y2, x1:x2]
        
        # Segment characters
        binary, gray = preprocess_plate_for_ocr(plate)
        char_images = segment_characters(binary, gray)
        
        if len(char_images) == 0:
            return {
                'success': False,
                'message': 'No characters detected',
                'image': img_rgb,
                'plate_bbox': best_detection['bbox']
            }
        
        # Recognize characters
        recognized_chars = self.recognize_characters(char_images)
        
        # Sort by position
        recognized_chars = sorted(recognized_chars, key=lambda x: x['position'])
        
        # Build license plate string
        plate_text = ''.join([c['char'] for c in recognized_chars])
        avg_confidence = np.mean([c['confidence'] for c in recognized_chars])
        
        result = {
            'success': True,
            'plate_text': plate_text,
            'confidence': avg_confidence,
            'num_chars': len(recognized_chars),
            'characters': recognized_chars,
            'plate_bbox': best_detection['bbox'],
            'plate_detection_conf': best_detection['confidence'],
            'image': img_rgb,
            'plate_image': cv2.cvtColor(plate, cv2.COLOR_BGR2RGB),
            'binary_image': binary
        }
        
        return result
    
    def visualize_result(self, result):
        """
        Visualize recognition result
        """
        if not result['success']:
            fig, ax = plt.subplots(1, 1, figsize=(10, 6))
            ax.imshow(result['image'])
            ax.set_title(f"❌ {result['message']}", fontsize=14, color='red')
            ax.axis('off')
            return fig
        
        # Create visualization
        fig = plt.figure(figsize=(15, 10))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # Original image with bbox
        ax1 = fig.add_subplot(gs[0, :])
        img_with_bbox = result['image'].copy()
        x1, y1, x2, y2 = result['plate_bbox']
        cv2.rectangle(img_with_bbox, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        # Add text
        plate_text = result['plate_text']
        conf = result['confidence']
        text = f"License Plate: {plate_text} (Conf: {conf:.2%})"
        cv2.putText(img_with_bbox, text, (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        
        ax1.imshow(img_with_bbox)
        ax1.set_title('Detection Result', fontsize=14, fontweight='bold')
        ax1.axis('off')
        
        # Cropped plate
        ax2 = fig.add_subplot(gs[1, 0])
        ax2.imshow(result['plate_image'])
        ax2.set_title('Cropped Plate', fontsize=12)
        ax2.axis('off')
        
        # Binary image
        ax3 = fig.add_subplot(gs[1, 1])
        ax3.imshow(result['binary_image'], cmap='gray')
        ax3.set_title('Binary Image', fontsize=12)
        ax3.axis('off')
        
        # Character confidence
        ax4 = fig.add_subplot(gs[1, 2])
        chars = [c['char'] for c in result['characters']]
        confs = [c['confidence'] for c in result['characters']]
        colors = ['green' if c > 0.9 else 'orange' if c > 0.7 else 'red' for c in confs]
        
        ax4.barh(range(len(chars)), confs, color=colors, alpha=0.7)
        ax4.set_yticks(range(len(chars)))
        ax4.set_yticklabels(chars, fontsize=12, fontweight='bold')
        ax4.set_xlabel('Confidence', fontsize=10)
        ax4.set_title('Character Confidence', fontsize=12)
        ax4.set_xlim([0, 1])
        ax4.grid(True, alpha=0.3)
        
        # Segmented characters
        ax5 = fig.add_subplot(gs[2, :])
        if len(result['characters']) > 0:
            # Show all characters
            char_imgs = []
            for i, char_data in enumerate(result['characters']):
                # Get original char image
                h, w = result['binary_image'].shape
                x, y, cw, ch = 0, 0, 28, 28  # Placeholder
                
                # Create labeled image
                char_img = np.ones((40, 28, 3), dtype=np.uint8) * 255
                char_img[6:34, :, 0] = result['binary_image'][::2, ::2] if result['binary_image'].shape[0] > 28 else cv2.resize(result['binary_image'], (28, 28))
                char_img[6:34, :, 1] = char_img[6:34, :, 0]
                char_img[6:34, :, 2] = char_img[6:34, :, 0]
                
                # Add label
                label = result['characters'][i]['char']
                conf = result['characters'][i]['confidence']
                cv2.putText(char_img, label, (8, 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 200, 0), 1)
                
                char_imgs.append(char_img)
            
            combined = np.hstack(char_imgs)
            ax5.imshow(combined)
            ax5.set_title(f'Recognized Characters: {plate_text}', fontsize=12, fontweight='bold')
        ax5.axis('off')
        
        return fig

# Create recognizer
recognizer = LicensePlateRecognizer(
    plate_detector=plate_detector,
    char_model=char_model,
    char_transform=char_transform,
    device=device
)

print("✅ License Plate Recognizer initialized")

NameError: name 'plate_detector' is not defined

In [ ]:
# Test on Sample Images 
# Get test images
test_images = list(Path('images/val').glob('*.jpg'))[:10]

print(f"🧪 Testing on {len(test_images)} images...\n")

results_list = []

for img_path in test_images:
    print(f"📸 Processing: {img_path.name}")
    
    result = recognizer.process_image(img_path)
    
    if result['success']:
        print(f"   ✅ Detected: {result['plate_text']}")
        print(f"   📊 Confidence: {result['confidence']:.2%}")
        print(f"   🔢 Characters: {result['num_chars']}")
    else:
        print(f"   ❌ {result['message']}")
    
    results_list.append(result)
    print()

# Visualize first few results
print("\n📊 Visualizing results...")
for i, result in enumerate(results_list[:3]):
    fig = recognizer.visualize_result(result)
    plt.savefig(f'result_{i+1}.png', dpi=150, bbox_inches='tight')
    plt.show()
    print(f"✅ Result {i+1} saved as 'result_{i+1}.png'")

🧪 Testing on 0 images...


📊 Visualizing results...


In [ ]:
# Batch Processing 
def batch_process_images(image_dir, output_csv='recognition_results.csv'):
    """
    Process all images in directory and save results
    """
    import pandas as pd
    from tqdm import tqdm
    
    image_files = list(Path(image_dir).glob('*.jpg')) + list(Path(image_dir).glob('*.png'))
    
    results_data = []
    
    print(f"🚀 Processing {len(image_files)} images...")
    
    for img_path in tqdm(image_files):
        result = recognizer.process_image(img_path, visualize=False)
        
        results_data.append({
            'filename': img_path.name,
            'success': result['success'],
            'plate_text': result.get('plate_text', ''),
            'confidence': result.get('confidence', 0.0),
            'num_chars': result.get('num_chars', 0),
            'message': result.get('message', '')
        })
    
    # Save to CSV
    df = pd.DataFrame(results_data)
    df.to_csv(output_csv, index=False)
    
    # Statistics
    success_rate = df['success'].sum() / len(df) * 100
    avg_confidence = df[df['success']]['confidence'].mean()
    
    print(f"\n📊 BATCH PROCESSING RESULTS:")
    print(f"   Total images: {len(df)}")
    print(f"   Successful: {df['success'].sum()}")
    print(f"   Failed: {(~df['success']).sum()}")
    print(f"   Success rate: {success_rate:.2f}%")
    print(f"   Avg confidence: {avg_confidence:.2%}")
    print(f"\n✅ Results saved to: {output_csv}")
    
    return df

# Process validation set
results_df = batch_process_images('images/val', output_csv='val_recognition_results.csv')

🚀 Processing 0 images...


0it [00:00, ?it/s]


KeyError: 'success'

In [ ]:
# Analyze Results
import pandas as pd

# Load results
df = pd.read_csv('val_recognition_results.csv')

# Overall statistics
print("📊 RECOGNITION STATISTICS")
print("="*70)
print(f"\nTotal Images: {len(df)}")
print(f"Successful Detections: {df['success'].sum()} ({df['success'].sum()/len(df)*100:.2f}%)")
print(f"Failed Detections: {(~df['success']).sum()} ({(~df['success']).sum()/len(df)*100:.2f}%)")

# Confidence distribution
successful_df = df[df['success']]
if len(successful_df) > 0:
    print(f"\nConfidence Statistics:")
    print(f"   Mean: {successful_df['confidence'].mean():.4f}")
    print(f"   Median: {successful_df['confidence'].median():.4f}")
    print(f"   Std: {successful_df['confidence'].std():.4f}")
    print(f"   Min: {successful_df['confidence'].min():.4f}")
    print(f"   Max: {successful_df['confidence'].max():.4f}")
    
    # Character count distribution
    print(f"\nCharacter Count Distribution:")
    char_counts = successful_df['num_chars'].value_counts().sort_index()
    for count, freq in char_counts.items():
        print(f"   {count} chars: {freq} images ({freq/len(successful_df)*100:.1f}%)")

# Visualize distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confidence distribution
if len(successful_df) > 0:
    axes[0].hist(successful_df['confidence'], bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0].axvline(successful_df['confidence'].mean(), color='red', linestyle='--', 
                    label=f"Mean: {successful_df['confidence'].mean():.3f}")
    axes[0].set_xlabel('Confidence', fontsize=12)
    axes[0].set_ylabel('Frequency', fontsize=12)
    axes[0].set_title('Confidence Distribution', fontsize=14, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# Character count distribution
if len(successful_df) > 0:
    char_counts.plot(kind='bar', ax=axes[1], color='lightgreen', edgecolor='black', alpha=0.7)
    axes[1].set_xlabel('Number of Characters', fontsize=12)
    axes[1].set_ylabel('Frequency', fontsize=12)
    axes[1].set_title('Character Count Distribution', fontsize=14, fontweight='bold')
    axes[1].grid(True, alpha=0.3, axis='y')
    axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.savefig('recognition_statistics.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Statistics visualization saved as 'recognition_statistics.png'")

EmptyDataError: No columns to parse from file

In [ ]:
# Error Analysis 
# Analyze failed cases
failed_df = df[~df['success']]

if len(failed_df) > 0:
    print("🔍 ERROR ANALYSIS")
    print("="*70)
    print(f"\nFailed Cases: {len(failed_df)}")
    
    # Failure reasons
    print("\nFailure Reasons:")
    failure_reasons = failed_df['message'].value_counts()
    for reason, count in failure_reasons.items():
        print(f"   {reason}: {count} ({count/len(failed_df)*100:.1f}%)")
    
    # Visualize some failed cases
    num_show = min(6, len(failed_df))
    if num_show > 0:
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        axes = axes.flatten()
        
        for idx, (_, row) in enumerate(failed_df.head(num_show).iterrows()):
            img_path = Path('images/val') / row['filename']
            img = cv2.imread(str(img_path))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            axes[idx].imshow(img_rgb)
            axes[idx].set_title(f"{row['filename']}\n{row['message']}", fontsize=10, color='red')
            axes[idx].axis('off')
        
        plt.suptitle('Failed Recognition Cases', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig('failed_cases.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print("\n✅ Failed cases visualization saved as 'failed_cases.png'")

In [ ]:
# Create Inference Function for New Images 
def recognize_license_plate(image_path, save_result=True, output_dir='output'):
    """
    High-level function to recognize license plate from image
    
    Parameters:
    -----------
    image_path: str or Path
        Path to input image
    save_result: bool
        Whether to save visualization
    output_dir: str
        Directory to save results
        
    Returns:
    --------
    dict: Recognition result
    """
    # Create output directory
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    # Process image
    result = recognizer.process_image(image_path)
    
    # Print result
    print(f"\n{'='*70}")
    print(f"📸 Image: {Path(image_path).name}")
    print(f"{'='*70}")
    
    if result['success']:
        print(f"✅ License Plate Detected: {result['plate_text']}")
        print(f"📊 Overall Confidence: {result['confidence']:.2%}")
        print(f"🔢 Number of Characters: {result['num_chars']}")
        print(f"📦 Plate Detection Confidence: {result['plate_detection_conf']:.2%}")
        print(f"\n📝 Character Details:")
        for i, char_data in enumerate(result['characters'], 1):
            print(f"   {i}. '{char_data['char']}' - Confidence: {char_data['confidence']:.2%}")
    else:
        print(f"❌ Recognition Failed: {result['message']}")
    
    # Save visualization
    if save_result:
        fig = recognizer.visualize_result(result)
        output_path = output_dir / f"{Path(image_path).stem}_result.png"
        plt.savefig(output_path, dpi=150, bbox_inches='tight')
        plt.close(fig)
        print(f"\n💾 Result saved to: {output_path}")
    
    return result

# Example usage
test_image = list(Path('images/val').glob('*.jpg'))[0]
result = recognize_license_plate(test_image, save_result=True, output_dir='inference_results')

In [ ]:
# Real-time Video Processing (Optional) 
def process_video(video_path, output_path='output_video.mp4', display=True):
    """
    Process video for license plate recognition
    
    Parameters:
    -----------
    video_path: str
        Path to input video
    output_path: str
        Path to save output video
    display: bool
        Whether to display video while processing
    """
    import cv2
    
    cap = cv2.VideoCapture(str(video_path))
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    print(f"🎥 Processing video...")
    print(f"   Resolution: {width}x{height}")
    print(f"   FPS: {fps}")
    print(f"   Total frames: {total_frames}")
    
    frame_count = 0
    detected_plates = []
    
    from tqdm import tqdm
    pbar = tqdm(total=total_frames)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_